In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import ee
ee.Initialize()

In [ ]:
def _merge_hist(current, previous):
    """merge two dictionaries by summing counts for each confidence key."""
    current = ee.Dictionary(current)
    previous = ee.Dictionary(previous)
    keys = current.keys()
    def _accumulate(key, acc):
        acc = ee.Dictionary(acc)
        k = ee.String(key)
        # sum the counts, defaulting to 0 if key missing
        return acc.set(k,
                       ee.Number(acc.get(k, 0))
                       .add(ee.Number(current.get(k))))
    return ee.List(keys).iterate(_accumulate, previous)

def summarize_alerts(name,
                     collection,
                     start_date,
                     end_date,
                     geom,
                     scale,
                     confidence_threshold,
                     confidence_band='confidence'):
    """
    Returns a single ee.Feature with properties:
      - 'dataset': the dataset name
      - 'totalAlerts': total pixels >= threshold across all images
      - 'confidence_histogram': merged histogram of confidence values
    """

    filtered = collection.filterDate(start_date, end_date).filterBounds(geom)

    def per_image(image):
        conf = image.select(confidence_band)
        masked = conf.updateMask(conf.gte(confidence_threshold))
        hist = masked.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=geom,
            scale=scale,
            maxPixels=1e13,
            bestEffort=True
        ).get(confidence_band)
        hist = ee.Dictionary(hist)
        return ee.Feature(None, {'confidence_histogram': hist})

    stats_fc = filtered.map(per_image)

    hist_list = stats_fc.aggregate_array('confidence_histogram')
    summed = ee.Dictionary(
        ee.List(hist_list).iterate(_merge_hist, ee.Dictionary({}))
    )

    total = ee.Number(summed.values().reduce(ee.Reducer.sum()))

    return ee.Feature(None, {
        'dataset': name,
        'totalAlerts': total,
        'confidence_histogram': summed
    })


In [22]:
gaul_id = "projects/ee-andyarnellgee/assets/crosscutting/GAUL_2024_L0_simplify_0_001deg_dice10k"
aoi = ee.FeatureCollection(gaul_id).filter(ee.Filter.eq("iso3_code", "COL"))

snpp = ee.ImageCollection("NASA/LANCE/SNPP_VIIRS/C2") 
noaa20 = ee.ImageCollection("NASA/LANCE/NOAA20_VIIRS/C2")
modis = ee.ImageCollection("FIRMS")

In [23]:
start_date = "2024-01-01"
end_date = "2024-01-02"

In [24]:
# Modis Colombia

# get the number of alerts in the area of interest
modis_alerts = modis.filterBounds(aoi).filterDate(start_date, end_date)



In [25]:
confidence_band = "confidence"
confidence_threshold = 1

collections = {
    'snpp':   ee.ImageCollection('NASA/LANCE/SNPP_VIIRS/C2'),
    # 'noaa20': ee.ImageCollection('NASA/LANCE/NOAA20_VIIRS/C2'),
    # 'modis':  ee.ImageCollection('FIRMS')
}


In [26]:
features = []
for name, collection in collections.items():
    feat = summarize_alerts(
        name, collection,
        start_date, end_date,
        aoi, 1000,
        confidence_threshold=1
    )
    features.append(feat)

# Combine into a single FeatureCollection
summary_fc = ee.FeatureCollection(features)
summary_fc = summary_fc.getInfo()
summary_fc

{'type': 'FeatureCollection',
 'columns': {'dataset': 'String',
  'system:index': 'String',
  'totalAlerts': 'Float'},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0',
   'properties': {'dataset': 'snpp', 'totalAlerts': 46}}]}

In [31]:
from component.scripts.gee import summarize_alerts
from component.parameter.app import GEE_SOURCES


In [44]:
alerts = summarize_alerts(
    name="MODIS",
    start_date="2024-01-01",
    end_date="2024-01-02",
    geom=aoi.geometry(),
    confidence_threshold=0,
)
alerts.getInfo()

{'type': 'Feature',
 'geometry': None,
 'properties': {'confidence_histogram': {'28': 15,
   '33': 2,
   '37': 27,
   '39': 9,
   '40': 3,
   '46': 4,
   '56': 21,
   '57': 24,
   '59': 10,
   '62': 9,
   '67': 3,
   '68': 15,
   '69': 27,
   '73': 6,
   '75': 12,
   '81': 6,
   '83': 15},
  'dataset': 'MODIS',
  'total_alerts': 208}}

In [61]:
modis_ic = ee.ImageCollection("FIRMS").filterDate("2024-01-01", "2024-01-30")
alert_raster = ee.Image(modis_ic.reduce(ee.Reducer.sum()).select("confidence_sum"))

In [62]:
bounding_boxes = alert_raster.clip(aoi.geometry()).reduceToVectors(
    # reducer=ee.Reducer.sum(),
    geometry=aoi.geometry(),
    scale=GEE_SOURCES["MODIS"]["scale"],
    geometryType="bb",
    eightConnected=True,
    maxPixels=1e13,
)

bounding_boxes.size().getInfo()

6453